In [ ]:
import os
import sys
sys.path.append(os.path.abspath('..'))
from config import *

!cp ../data/acorns.clean.csv .
!cp ../data/acorns.csv .

# グループ化処理

データに複数のグループが含まれており、それぞれのグループに対して同じような処理を行いたい場合は、Pandas の `groupby` メソッドを利用すると効率的です。本節では、どんぐりのデータセット（acorns.csv）を使って、グループ化処理の基本的な使い方を紹介していきます。

In [ ]:
x = pd.read_csv('acorns.clean.csv')
x

## groupby

データフレームのある列の値に基づいて、データを一時的に複数のグループ（サブセット）に分けて処理を行うとき、`groupby` メソッドを使用します。`groupby` メソッドを利用するとき、グループ化の基準となる列名を指定します。たとえば、どんぐりのデータセットにおいて、樹種（tree 列の値）ごとにデータをグループ化したい場合、次のように記述します。

In [ ]:
x_subsets = x.groupby('tree')

`groupby` メソッドを使うと、指定した列の値に基づいてデータをいくつかのグループに分けることができます。`groupby` が返すオブジェクトは、グループ化されたサブセットの集まりで、リストのように扱うことが可能です。このオブジェクトを `for` 文で繰り返し処理することで、各グループに対して同じ処理を適用することができます。ここで注意すべきなのは、`groupby` オブジェクトは各グループを「グループ名」と「そのグループのサブセット（データフレーム）」のペアで保持しているという点です。そのため、`for` 文では 2 つの変数を用意して、それぞれを受け取る必要があります。

次は、各樹種のどんぐりについて、重さ（weight）の平均値を計算する例です。`groupby` の結果に対して `for` 文で繰り返し処理を行います。各繰り返しでは、グループ名（`tree`）とそのグループに対応するデータ（`x_subset`）が得られるため、それを使って平均値を計算しています。

In [ ]:
for tree, x_subset in x_subsets:
    print(tree)
    print(x_subset['weight'].mean())

また、次のように `for` 文の中で `groupby` を直接利用することもできます。

In [ ]:
for tree, x_sub in x.groupby('tree'):
    print(tree)
    print(x_sub['weight'].mean())

次に、重さ（weight）だけでなく、高さ（height）、直径（diameter）の 3 つの項目すべてに対して、樹種ごとの平均値を計算する例を示します。

In [ ]:
for tree, x_sub in x.groupby('tree'):
    print(tree)
    print(x_sub[['weight', 'height', 'diameter']].mean())

`groupby` の結果を `for` 構文で処理すると、各グループごとの結果はシリーズ型で出力され、一覧性に欠けて見づらくなります。そこで、集計結果を見やすくするために、各グループに対する計算結果を一時的にリスト `outputs` に追加し、最後にそれをデータフレームに変換します。

In [ ]:
outputs = []

for tree, x_sub in x.groupby('tree'):
    x_table = x_sub[['weight', 'height', 'diameter']].mean()
    x_table['tree'] = tree
    outputs.append(x_table)

outputs = pd.DataFrame(outputs)
outputs

次に、もう少し複雑な処理の例を見てみましょう。樹種ごとに、どんぐりの重さの最大値・最小値、さらにその差（範囲）を計算します。各グループの計算結果は、後でデータフレームに変換しやすいように、辞書形式で保存し、それをリスト `outputs` に追加していきます。

In [ ]:
outputs = []

for tree, x_sub in x.groupby('tree'):
    x_sub_max = x_sub['weight'].max()
    x_sub_min = x_sub['weight'].min()
    outputs.append({
        'tree': tree,
        'weight_max': x_sub_max,
        'weight_min': x_sub_min,
        'weight_range': x_sub_max - x_sub_min
    })

outputs = pd.DataFrame(outputs)
outputs

このように、`groupby` を使えば、グループ単位で柔軟な集計処理を行うことができます。必要に応じて、集計関数を自作したり、複数の列や複雑な計算を加えることも可能です。

## agg

基本的な集計処理では、`groupby` と `agg` メソッドを組み合わせて使うことで、簡潔に記述できます。たとえば、各樹種（tree）ごとにどんぐりの各特徴量の最大値と最小値を集計するには、次のように `agg` メソッドに集計関数をリスト形式で渡します。なお、最大値や最小値を計算する関数は自作してもかまいませんが、ここでは NumPy の `np.max` と `np.min` を利用しています。

In [ ]:
import numpy as np

x.groupby('tree').agg([np.max, np.min])

標準で用意されていない集計処理を行いたい場合は、ユーザー独自の関数を定義し、それを `agg` メソッドに渡すことができます。例えば、「最大値と最小値の差（範囲）」を計算する場合は、次のようにします。

In [ ]:
def calc_range(x):
    return x.max() - x.min()

x.groupby('tree').agg([np.max, np.min, calc_range])

特定の列だけを対象に集計したい場合は、`groupby` のあとで該当する列を選択してから `agg` メソッドを適用します。例えば、どんぐりの重さ（weight）と高さ（height）のみに対して、それぞれ平均（`np.mean`）と分散（`np.var`）を計算するには、次のように書きます。

In [ ]:
x.groupby('tree')[['weight', 'height']].agg(['mean', 'var'])

以上のように、`agg` メソッドを使うことで、コードを簡潔に記述できるだけでなく、グループごとの統計量を効率的にまとめて計算できるようになります。

これなら、冬眠前のどんぐり棚卸しも一瞬で終わりますね。

## apply

`groupby` の後には、`apply` メソッドを使ってグループごとの処理を行うこともできます。`agg` メソッドは、各グループに対して平均、合計、最大値、最小値といった定型的な集計処理を簡潔に記述でき、高速に動作するのが特徴です。一方で、`apply` メソッドはより柔軟で強力な処理が可能です。グループごとのサブセットに対して、欠損値の補完、並べ替え、フィルタリング、標準化など、自由な処理を適用することができます。基本的には、`for` 文を使ってグループごとに処理を書くのと同じことを、簡潔に効率よく実行できます。

なお、`apply` で実現できる処理は多くの場合 `for` 文でも代替可能です。そのため、ここではいくつかの使用例を示し、深入りしません。

In [ ]:
x.groupby('tree').apply(np.mean)

In [ ]:
x.groupby('tree')['weight'].apply(np.max)

In [ ]:
def calc_range(x):
    return x.max() - x.min()

x.groupby('tree')['weight'].apply(calc_range)


```{admonition} 練習問題 PG-1
acorns.csv データを読み込み、どんぐり各種の重さ（weight）の平均と分散を求めなさい。
```

```{admonition} 練習問題 PG-2
acorns.csv データを読み込み、どんぐり各種の重さ（weight）、高さ（height）、直径（diameter）の平均値と分散をそれぞれ求めなさい。
```

```{admonition} 練習問題 PG-3
データ全体から計算した平均と標準偏差の「平均 ± 2 × 標準偏差」の範囲に含まれないデータを外れ値と定義します。acorns.csv データを読み込み、どんぐり各種に対して、重さ（weight）が外れ値にあたるどんぐりの個数を求めよ。
```

In [ ]:
!rm acorns.clean.csv
!rm acorns.csv